<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/AskAPatient/Experiment%20(Change%20testing%20dataset)/Codes/ask_pubmed_tweet_overlap_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [21]:
if os.path.exists('medical-concept-normalization'):
  print('folder exist')
else:
  os.mkdir(base_dir)

folder exist


In [22]:
if os.path.exists('medical-concept-normalization/data_collection'):
  print('folder exist')
else:
  os.mkdir('medical-concept-normalization/data_collection')

folder exist


**Install fastai and clone the fastai for text classification**



In [23]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [24]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

fatal: destination path 'demo-nlp-classification-fastai' already exists and is not an empty directory.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# datasets

train_csv = 'https://raw.githubusercontent.com/unt-iialab/medical-concept-normalization/master/AskAPatient/Experiment%20(Change%20testing%20dataset)/Training%20set/AskAPatient.fold-0.train.csv'
valid_csv = 'https://raw.githubusercontent.com/unt-iialab/medical-concept-normalization/master/AskAPatient/Experiment%20(Change%20testing%20dataset)/Validation%20set/AskAPatient.fold-0.validation.csv'
test_csv = 'https://raw.githubusercontent.com/unt-iialab/medical-concept-normalization/master/AskAPatient/Experiment%20(Change%20testing%20dataset)/Testing%20set/AskAPatient_overlap_0.csv'
fine_tuning_data = 'fine_tuning_language_model_pubmed_bigtweet_mixed.csv'

In [57]:
# training data set
train = pd.read_csv(train_csv, header=None)
print(train.head())

# validating data set
valid = pd.read_csv(valid_csv,header=None)
# valid.head()

# testing data set
test = pd.read_csv(test_csv,header=None)
# test.head()

   0                1
0  0             Fear
1  0             Fear
2  0             Fear
3  0  scared to death
4  1   Abdominal rash


In [58]:
# Creating DataBunch for language modelling
data_lm = TextLMDataBunch.from_csv(path, fine_tuning_data)

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [59]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [60]:
data_clas.add_test(test,label=0)

In [0]:
data_lm.save()
data_clas.save()

In [0]:
data_lm.train_ds.vocab.itos

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [65]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.190758,4.629697,0.251497,21:47


In [66]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.432345,4.282485,0.284477,21:44


In [67]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.115665,4.243641,0.288160,21:39


In [68]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,4.070340,4.213881,0.291220,21:46


In [0]:
learn.save_encoder('tuning_pubmed_bigtweet_overlap_0_first')

In [70]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('tuning_pubmed_bigtweet_overlap_0_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.644461,3.591742,0.423669,02:48


In [71]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.715802,3.108843,0.491124,02:30


In [72]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.394318,2.853024,0.525444,03:17


In [73]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.158136,2.552487,0.564497,03:22
1,1.900518,2.452618,0.586982,03:08


In [74]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.824951,2.303917,0.603550,03:12
1,1.681460,2.205513,0.626036,03:30


In [75]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.594424,2.120790,0.628402,03:31
1,1.452530,2.082484,0.636686,03:15


In [76]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.391755,2.090409,0.639053,03:24
1,1.327287,2.019941,0.640237,03:34


In [77]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.303346,1.979786,0.646154,03:22
1,1.214436,1.942078,0.648521,03:09


In [78]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.194643,1.947708,0.656805,03:09
1,1.197459,1.930854,0.665089,03:30
2,1.088198,1.886148,0.673373,03:12
3,1.016176,1.884116,0.669823,03:22


In [79]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.002452,1.903113,0.673373,03:27
1,1.063407,1.913070,0.671006,03:29
2,1.001928,1.863434,0.678106,03:24
3,0.917419,1.840454,0.679290,03:34


In [80]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.034596,1.914115,0.676923,03:08
1,0.971052,1.906000,0.678106,03:13
2,0.907905,1.923906,0.679290,03:05
3,0.896369,1.878990,0.680473,03:29


In [81]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.910520,1.924912,0.673373,03:03
1,0.922303,1.944099,0.667456,03:28
2,0.924059,1.904284,0.692308,03:26
3,0.816782,1.870267,0.693491,03:11


In [82]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.886043,1.906616,0.680473,03:17
1,0.916214,1.933653,0.680473,03:12
2,0.851328,1.963483,0.684024,03:09
3,0.782646,1.895140,0.680473,03:25


In [83]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.823863,1.965859,0.673373,03:24
1,0.830817,1.938711,0.686391,03:28
2,0.802777,1.981095,0.688757,03:16
3,0.816232,1.971830,0.692308,03:24


In [84]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.811313,1.982760,0.665089,03:26
1,0.820887,1.942274,0.686391,03:20
2,0.816948,1.968326,0.682840,03:25
3,0.779242,1.972479,0.691124,03:28


In [85]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.797394,1.934597,0.679290,03:15
1,0.777109,1.968076,0.692308,03:28


In [86]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.869135,2.023431,0.678106,03:32
1,0.789028,1.940908,0.686391,03:10


In [87]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.788270,1.966304,0.685207,03:26


In [0]:
# save the best model

learn.save_encoder('tuning_pubmed_bigtweet_overlap_0_last')

# Part three: Predict on the test dataset

In [89]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [90]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

610
610
[840, 910, 245, 699, 5, 1026, 1026, 9, 10, 623, 438, 225, 135, 27, 597, 288, 29, 30, 211, 30, 289, 37, 880, 348, 77, 745, 1031, 1031, 289, 49, 245, 61, 729, 71, 71, 384, 260, 524, 470, 182, 864, 289, 289, 470, 80, 848, 89, 627, 289, 517, 94, 826, 318, 98, 492, 98, 49, 532, 402, 337, 100, 490, 809, 168, 214, 34, 108, 108, 636, 182, 313, 111, 470, 111, 111, 211, 873, 730, 967, 142, 470, 470, 125, 470, 125, 579, 129, 129, 289, 745, 180, 144, 727, 415, 172, 498, 157, 517, 165, 826, 168, 183, 190, 79, 182, 260, 363, 37, 318, 470, 470, 190, 190, 190, 190, 190, 190, 190, 197, 34, 214, 214, 214, 805, 245, 96, 470, 210, 210, 210, 210, 190, 214, 470, 419, 214, 100, 265, 470, 470, 220, 44, 228, 289, 289, 289, 318, 544, 541, 234, 574, 265, 386, 470, 470, 470, 557, 187, 470, 470, 470, 470, 470, 70, 318, 733, 173, 470, 245, 245, 846, 246, 298, 470, 470, 728, 259, 309, 190, 470, 265, 419, 214, 265, 289, 265, 214, 265, 199, 89, 627, 470, 728, 289, 359, 283, 52, 107, 419, 173, 289, 289, 289, 28

In [91]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

208
0.34098360655737703
